In [1]:
#Instalação
!pip install pymupdf wget

You should consider upgrading via the '/Users/carlosalberto/Documents/projects/ai/TEESI_2025/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import fitz
import os

import numpy as np
import pandas as pd

import re

In [3]:
#Loader PDF to Text
class PdfToText():
  def __init__(self):
    pass

  def convert_from_file(self, filename):
    try:
      doc = self.load_pdf(filename)
    except:
      print( "Unable to open file." )
      return ""

    text = ""
    i = 0;
    for page in doc:
      text += page.get_text()+f"pagina {i}" + (50 * "-") + "\n\n\n"
      i = i + 1;
    
    doc.close()
    return text

  def load_pdf(self, filename):
    return fitz.open(filename)

class EnemExam(PdfToText):
  def __init__(self):
    pass

  def read_questions(self, filename):
    self.text = self.convert_from_file(filename)
    #print(self.text)
    self.find_topics()
    self.find_questions()
    self.find_color()

  def find_topics(self):
    self._topics = ['matemática e suas tecnologias',
                    'linguagens, códigos',
                    'ciências humanas',
                    'ciências da natureza',
                    ]

    topic_poss = [ self.text.lower().rfind(topic) for topic in self._topics ]

    self._topics_loc = [ (topic,topic_pos) for topic, topic_pos in zip(self._topics, topic_poss) if topic_pos>0 ]
    #ordena pela posição do tópico no texto.
    self._topics_loc.sort( key=lambda t: t[1] )

  def find_color(self):
    self._color = ""
    padroes = [
        r"CADERNO\s*\n(\w+)",                    # Padrão 1: "CADERNO\nAZUL"
        r"Caderno\s+\d+\s*-\s*(\w+)\s*-"         # Padrão 2: "Caderno 1 - AZUL - ..."
    ]
    
    for padrao in padroes:
      match = re.search(padrao, self.text.lower(), re.IGNORECASE)
      if match:
        self._color = match.group(1).upper()

  def find_questions(self):
    matchs = []
    for match in re.finditer('\n(questão) [0-9]+', self.text.lower()):
      matchs.append( match.start() )

    start_pos = matchs[-1]
    match_end = re.search(r'\*[a-z0-9]+\*', self.text.lower()[start_pos:])
    matchs.append(start_pos + match_end.start())
   
    self.questions = []
    self.questions_topic = []
    self.questions_start = []
    for i in range( 1, len(matchs) ):
      self.questions.append( self.text[matchs[i-1]:matchs[i] ] )
      self.questions_topic.append( self.define_question_topic(matchs[i-1]) )
      self.questions_start.append( matchs[i-1] )

    
  def define_question_topic(self, start):
    question_topic = 0 #UNDEFINED

    for topic, topic_start in self._topics_loc:
      if start > topic_start:
        question_topic = (self._topics.index(topic)+1)

    return question_topic

In [13]:
nome_arquivo_provas = os.listdir('./temp/')
enem_exam = EnemExam()
enem_dfs = []

for nome_arquivo_prova in nome_arquivo_provas:
  enem_exam.read_questions('./temp/'+nome_arquivo_prova)
  enem_dfs.append(
      pd.DataFrame( data=dict( 
        enunciado=enem_exam.questions, 
        label=enem_exam.questions_topic, 
        cor_prova=enem_exam._color)
    )
  )

#Mapeie cada label para o nome do tópico
enem_dfs = [enem_df.replace({0:'desconhecido',1:'matemática e suas tecnologias', 2:'linguagens, códigos e suas tecnologias', 3:'ciências humanas e suas tecnologias', 4:'ciências da natureza e suas tecnologias'}) for enem_df in enem_dfs]
#Mostre uma posição específica do enem_dfs
print(enem_dfs[0].iloc[0])

enunciado    \nQUESTÃO 01\t\nAs my official bio reads, I wa...
label                                             desconhecido
cor_prova                                                 AZUL
Name: 0, dtype: object


In [14]:
from sklearn.model_selection import train_test_split

questions_df = pd.concat( enem_dfs )


train_df, test_df = train_test_split(questions_df, test_size=0.2, random_state=42)

print(len(train_df))
print(len(test_df))

76
19


In [15]:
train_df.to_json( 'data/enem/train/enem_questoes.json', orient='records' )
test_df.to_json( 'data/enem/test/enem_questoes.json', orient='records' )

In [16]:
from datasets import load_dataset

# Caminho para o diretorio
json_dir_path = "./data/enem/"

# Carregar o dataset a partir do diretorio
dataset = load_dataset("json", data_dir=json_dir_path)

# Exibir algumas amostras do dataset
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['enunciado', 'label', 'cor_prova'],
        num_rows: 76
    })
    test: Dataset({
        features: ['enunciado', 'label', 'cor_prova'],
        num_rows: 19
    })
})


In [17]:
import pandas as pd

dataset.set_format(type="pandas")
df = dataset["train"][:300]

#df.head()

print(df.iloc[0]["enunciado"])


QUESTÃO 62	
Brasil e Argentina chegaram a um acordo para a 
redução em 10% da Tarifa Externa Comum (TEC) do 
Mercosul. O consenso foi alcançado durante negociação 
entre o ministro das Relações Exteriores do Brasil e o 
seu equivalente argentino, no Palácio do Itamaraty, em 
Brasília, no início do mês de outubro de 2021. A redução 
da TEC é um antigo desejo do Brasil, que pretende 
abrir mais sua economia e, com isso, ajudar a controlar 
a inflação. Já a Argentina temia que a medida pudesse 
afetar sua produção industrial. O acordo vai abranger 
uma ampla gama de produtos e ainda será apresentado 
ao Paraguai e Uruguai, para que seja formalizado.
Brasil e Argentina fecham acordo para corte de 10% na tarifa do Mercosul. 
Disponível em: https://oglobo.globo.com. Acesso em: 8 out. 2021 (adaptado).
A necessidade de negociação diplomática para viabilizar 
o acordo tarifário mencionado é explicada pela seguinte 
característica do Mercosul:
A	
A	 Limitação da circulação financeira.
B	
B	 Pad